#### LeetCode - SQL - #607

Refer [607. Sales Person](https://leetcode.com/problems/sales-person/description/)

Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name  **"RED"**.

Return the result table in  **any order**.

In [ ]:
data_sales = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
columns_sales = ['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']
schema_sales = {'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'}

data_company = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
columns_company = ['com_id', 'name', 'city']
schema_company = {'com_id':'Int64', 'name':'object', 'city':'object'}

data_orders = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
columns_orders = ['order_id', 'order_date', 'com_id', 'sales_id', 'amount']
schema_orders = {'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'}

In [ ]:
import pandas

pandas_df_sales = pandas.DataFrame(data=data_sales, columns=columns_sales).astype(schema_sales)
pandas_df_company = pandas.DataFrame(data=data_company, columns=columns_company).astype(schema_company)
pandas_df_orders = pandas.DataFrame(data=data_orders, columns=columns_orders).astype(schema_orders)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

# Spark DataFrame using Pandas DataFrame
df_sales = spark_context.createDataFrame(pandas_df_sales)
df_sales.printSchema()
df_sales.show(5)

df_company = spark_context.createDataFrame(pandas_df_company)
df_company.printSchema()
df_company.show(5)

df_orders = spark_context.createDataFrame(pandas_df_orders)
df_orders.printSchema()
df_orders.show(5)

In [ ]:
df_sales_with_company_red \
    = df_orders \
        .join(df_company, (df_orders.com_id == df_company.com_id) & (df_company.name == 'RED'), "inner") \
        .select(df_orders.sales_id)

list_sales_with_company_red = list(df_sales_with_company_red.toPandas()["sales_id"])

In [ ]:
# Using '~' is used to negate the 'isin' condition; equivalent to 'NOT IN' in SQL
df_sales \
    .filter(~df_sales.sales_id.isin(list_sales_with_company_red)) \
    .select(df_sales.name) \
    .show()